Import packages

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

Import dataset and separate features from labels

In [6]:
path = r"C:\Users\User\Kaggle\Kaggle-main\Titanic\titanic" + "\\"
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")

train.describe()
test.describe()
print(train.isnull().sum())
print(test.isnull().sum())

train.info()
test.info()
  
X = train.iloc[:, 0]
X = pd.concat([X, train.iloc[:, 2:]], axis = 1)
y_train = train.iloc[:, 1].values

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64

D:\Anaconda\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


We can use Decision tree or Random Forest algorithm to replace NaNs in Age column

In [7]:
def decision_tree(X):
    lbe = LabelEncoder()
    X['Sex'] = lbe.fit_transform(X['Sex'])
    train = X.loc[~pd.isnull(X.iloc[:,3])].iloc[:,[2,4,5,8,9,10,11,12,13,14,15,16,17]]
    test = X.loc[pd.isnull(X.iloc[:,3])].iloc[:,[2,4,5,8,9,10,11,12,13,14,15,16,17]]
#     print(train)
#     print(test)
    ### manual creating of age labels and then using DecisionTreeClassifier to predict age
    # clf_tree = DecisionTreeClassifier(criterion='entropy', random_state=1)
    # res = []
    # for i in range(0,X['Age'].size):
    #     if X['Age'][i]>0 and X['Age'][i]<19:
    #         res.append(18)
    #     elif X['Age'][i]>18 and X['Age'][i]<26:
    #         res.append(25)
    #     elif X['Age'][i]>25 and X['Age'][i]<36:
    #         res.append(35)
    #     elif X['Age'][i]>35 and X['Age'][i]<50:
    #         res.append(50)
    #     else:
    #         res.append(51)
    # # обучаем дерево
    # print(res)
    # X['Age_Labels'] = res
    # clf_tree = DecisionTreeRegressor(random_state=2)
    # clf_tree.fit(train, X.loc[~pd.isnull(X.iloc[:,4])].iloc[:,11])
    # print(clf_tree.predict(test))
    # RESULT of DecisionTreeClassifier is the same as DecisionTreeRegressor, but needs much more affords

    reg_tree = DecisionTreeRegressor(random_state=2)
    reg_tree.fit(train, X.loc[~pd.isnull(X.iloc[:,3])].iloc[:,3])
    result = reg_tree.predict(test)
#     print(reg_tree.predict(test))
    return result

In [82]:
"""
My own manual replacing NaNs from age
"""
def remove_nan_from_age(frame_orig, name_col_numb, sex_col_numb, Age_col_numb, sibsp_col_numb, parch_col_numb):
    frame = frame_orig
    cluster = []
    for i in range(0,frame['Ticket'].size):
        if frame.iloc[i, sibsp_col_numb] == 0 and frame.iloc[i, parch_col_numb] == 0:
            cluster.append(0)
        elif frame.iloc[i, sex_col_numb] == 'female' and '(' in frame.iloc[i, name_col_numb] and frame.iloc[i, sibsp_col_numb] > 0 and frame.iloc[i, parch_col_numb] > 0:
            cluster.append(1)
        elif frame.iloc[i, sex_col_numb] == 'female' and '(' not in frame.iloc[i, name_col_numb] and frame.iloc[i, parch_col_numb] > 1:
            cluster.append(2)
        elif frame.iloc[i, sex_col_numb] == 'female' and '(' not in frame.iloc[i, name_col_numb] and frame.iloc[i, parch_col_numb] == 1:
            cluster.append(3)
        elif frame.iloc[i, sex_col_numb] == 'female' and '(' not in frame.iloc[i, name_col_numb] and frame.iloc[i, parch_col_numb] == 0 and frame.iloc[i, sibsp_col_numb] > 0:
            cluster.append(4)
        elif frame.iloc[i, sex_col_numb] == 'male' and frame.iloc[i, sibsp_col_numb] == 0 and frame.iloc[i, parch_col_numb] == 1:
            cluster.append(5)
        elif frame.iloc[i, sex_col_numb] == 'male' and frame.iloc[i, sibsp_col_numb] == 0 and frame.iloc[i, parch_col_numb] > 1:
            cluster.append(6)
        elif frame.iloc[i, sex_col_numb] == 'male' and frame.iloc[i, sibsp_col_numb] > 0 and frame.iloc[i, parch_col_numb] == 0:
            cluster.append(7)
        elif frame.iloc[i, sex_col_numb] == 'male' and frame.iloc[i, sibsp_col_numb] == 1 and frame.iloc[i, parch_col_numb] > 0:
            cluster.append(8)
        elif frame.iloc[i, sex_col_numb] == 'male' and frame.iloc[i, sibsp_col_numb] > 1 and frame.iloc[i, parch_col_numb] > 0:
            cluster.append(9)
        else:
            cluster.append(10)
    frame['cluster'] = cluster
    cluster_median = frame.groupby('cluster')['Age'].median().reset_index() 
    for i in range(0,frame['Ticket'].size):
        if not frame.iloc[i,Age_col_numb] > -1:
            frame.iloc[i,Age_col_numb] = cluster_median.iloc[frame.iloc[i,frame.shape[1] - 1],1]
    return frame
            

Preprocessing datasets function

In [8]:
def preprocessing(train):
    """
    Clearing 'Ticket' column
    """
    res = []
    for i in range(0,train['Ticket'].size):
        if [int(j) for j in train.loc[i,'Ticket'].split() if j.isdigit()] == []:
            res.append(None)
        else:
            res.append([int(j) for j in train.loc[i,'Ticket'].split() if j.isdigit()][0])
    train["clear_ticket"] = res
    
    """
    Clearing passenger's Status
    """
    status = []
    for i in range(0,train['Ticket'].size):
        status.append((train.loc[i,'Name'].split(", "))[1].split(".")[0])
    train["clear_status"] = status


    train = train.fillna(value = {'clear_ticket': 0, 'Embarked': 0})
    """
    Get dummies from Categorical variables
    """
    from sklearn.preprocessing import LabelEncoder
    import pandas as pd

    lbe = LabelEncoder()
    train['Sex'] = lbe.fit_transform(train['Sex'])
    dummies = pd.get_dummies(train.loc[:, ['Embarked', 'clear_status']], prefix = ['Embarked', 'Status'])
    train = pd.concat([train, dummies], axis = 1)
    train = train[['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Fare', 'clear_ticket', 'Embarked_C', 'Embarked_Q', 'Status_Col', 'Status_Dr',
       'Status_Master', 'Status_Miss', 'Status_Mr', 'Status_Mrs',
       'Status_Ms', 'Status_Rev']]
    """
    Remove NaN ages
    """
    dec_tree_age = decision_tree(train)
    j = 0
    for i in range(0, train['clear_ticket'].size):
        if train['Age'][i] == np.nan:
            train['Age'][i] = dec_tree_age[j]
            j = j + 1
    #train = remove_nan_from_age(train, 3, 4, 5, 6, 7)  
    train = train.fillna(train.mean())  
    return train
    
    
 

Split dataframe on test and train datasets

In [9]:
X_train = preprocessing(X)
X_test = preprocessing(test)

print(X_train)
print(X_test)
X_train_model, X_test_model, y_train_model, y_test_model = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)




"""
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)X_test = sc_X.transform(X_test)sc_y = StandardScaler()y_train = sc_y.fit_transform(y_train)
res = [int(i) for i in test_string.split() if i.isdigit()]
imputer = Imputer(missing_values = "NaN", strategy = "mean", axis = 0)
imputer = imputer.fit(train.iloc[:, [5, 9]])
train.iloc[:, [5, 9]] = imputer.transform(train.iloc[:, [5, 9]])
"""

     PassengerId  Pclass  Sex        Age  SibSp  Parch      Fare  \
0              1       3    1  22.000000      1      0    7.2500   
1              2       1    0  38.000000      1      0   71.2833   
2              3       3    0  26.000000      0      0    7.9250   
3              4       1    0  35.000000      1      0   53.1000   
4              5       3    1  35.000000      0      0    8.0500   
5              6       3    1  29.699118      0      0    8.4583   
6              7       1    1  54.000000      0      0   51.8625   
7              8       3    1   2.000000      3      1   21.0750   
8              9       3    0  27.000000      0      2   11.1333   
9             10       2    0  14.000000      1      0   30.0708   
10            11       3    0   4.000000      1      1   16.7000   
11            12       1    0  58.000000      0      0   26.5500   
12            13       3    1  20.000000      0      0    8.0500   
13            14       3    1  39.000000      1 

'\nsc_X = StandardScaler()\nX_train = sc_X.fit_transform(X_train)X_test = sc_X.transform(X_test)sc_y = StandardScaler()y_train = sc_y.fit_transform(y_train)\nres = [int(i) for i in test_string.split() if i.isdigit()]\nimputer = Imputer(missing_values = "NaN", strategy = "mean", axis = 0)\nimputer = imputer.fit(train.iloc[:, [5, 9]])\ntrain.iloc[:, [5, 9]] = imputer.transform(train.iloc[:, [5, 9]])\n'

Build a model

In [10]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train_model, y_train_model)


cm = confusion_matrix(y_test_model, classifier.predict(X_test_model))
print(cm)

[[101   9]
 [ 44  25]]


We can try DecisionTreeCLassifier instead of LogisticRegression, it could give better result, cause LogisticRegression works not very good with a lot of Labeled features

In [14]:
rnd_forest = RandomForestClassifier(n_estimators = 300, random_state = 0)
rnd_forest.fit(X_train_model, y_train_model)


cm = confusion_matrix(y_test_model, rnd_forest.predict(X_test_model))
print(cm)


[[102   8]
 [ 17  52]]


Predict test dataset and write it inti csv file

In [28]:
predicted_labels = rnd_forest.predict(X_test)
result = pd.DataFrame(test['PassengerId'])
result["Survived"] = pd.Series(predicted_labels)
# result.insert(loc=1, column='Survived', value=pd.Series(predicted_labels))
print(result)
result.to_csv('Survived.csv')

                                                PassengerId  Survived
0                                                       892       0.0
1                                                       893       0.0
2                                                       894       0.0
3                                                       895       0.0
4                                                       896       1.0
5                                                       897       0.0
6                                                       898       0.0
7                                                       899       0.0
8                                                       900       1.0
9                                                       901       0.0
10                                                      902       0.0
11                                                      903       0.0
12                                                      904       1.0
13                  